# 📥 Collecte et Mise en Cache des Données - Ligue 1

Ce notebook s'occupe de **récupérer et mettre en cache** toutes les données nécessaires pour les prédictions.

## 🎯 Objectif

Séparer la logique de collecte de données de la logique de prédiction pour :
- ✅ **Économiser les appels API** : Une seule exécution met tout en cache
- ✅ **Accélérer les prédictions** : Les autres notebooks utilisent le cache
- ✅ **Clarté** : Séparation des responsabilités
- ✅ **Contrôle** : Choisir quand rafraîchir les données

## 📋 Données collectées

1. **Liste des équipes** de Ligue 1
2. **Statistiques détaillées** de chaque équipe
3. **Classement** de la Ligue 1
4. **Historique head-to-head** des principales confrontations

## ⏱️ Fréquence recommandée

Exécutez ce notebook :
- **Avant chaque journée** : Pour avoir les données à jour
- **Après les matchs** : Pour mettre à jour les statistiques
- **Maximum 1 fois par jour** : Pour économiser le quota API (100 requêtes/jour)

## 📊 Estimation des appels API

- Liste des équipes : **1 appel**
- Statistiques (20 équipes) : **20 appels**
- Classement : **1 appel**
- Head-to-head (optionnel) : **~10 appels**

**Total : environ 32 appels API** (33% du quota quotidien)

In [12]:
# Installation des dépendances (si nécessaire)
# !pip install requests pandas python-dotenv

import requests
import json
import pandas as pd
import os
from dotenv import load_dotenv
import time
import hashlib
from datetime import datetime

print("✅ Bibliothèques importées")

✅ Bibliothèques importées


In [13]:
# Configuration - Chargement depuis .env
load_dotenv()

API_KEY = os.getenv("API_FOOTBALL_KEY", "votre_clé_api_ici")
LIGUE1_ID = int(os.getenv("LIGUE1_ID", 61))
CURRENT_SEASON = int(os.getenv("CURRENT_SEASON", 2025))

print(f"✅ Configuration chargée : Ligue {LIGUE1_ID}, Saison {CURRENT_SEASON}")
print(f"📅 Saison {CURRENT_SEASON}-{CURRENT_SEASON+1}")

if API_KEY == "votre_clé_api_ici":
    print("⚠️ N'oubliez pas de configurer votre API_FOOTBALL_KEY dans le fichier .env")

✅ Configuration chargée : Ligue 61, Saison 2025
📅 Saison 2025-2026


## 🔧 Classe FootballAPI

Avec système de cache sur disque - durée prolongée pour éviter les rafraîchissements inutiles.

In [27]:
class FootballAPI:
    """Classe pour interagir avec l'API-Football avec système de cache sur disque"""
    
    def __init__(self, api_key, league_id=61, season=2025, cache_duration=86400, cache_dir='cache'):
        self.base_url = "https://v3.football.api-sports.io"
        self.headers = {'x-apisports-key': api_key}
        self.league_id = league_id
        self.season = season
        self.cache_duration = cache_duration  # 24h par défaut pour collecte
        self.cache_dir = cache_dir
        self.api_calls = 0
        
        if not os.path.exists(self.cache_dir):
            os.makedirs(self.cache_dir)
            print(f"📁 Dossier cache créé: {self.cache_dir}")
    
    def _get_cache_key(self, endpoint, params):
        params_str = '&'.join([f"{k}={v}" for k, v in sorted(params.items())])
        return f"{endpoint}?{params_str}"
    
    def _get_cache_filename(self, cache_key):
        hash_key = hashlib.md5(cache_key.encode()).hexdigest()
        return os.path.join(self.cache_dir, f"{hash_key}.json")
    
    def _get_from_cache(self, cache_key):
        cache_file = self._get_cache_filename(cache_key)
        
        if os.path.exists(cache_file):
            file_age = time.time() - os.path.getmtime(cache_file)
            
            if file_age < self.cache_duration:
                try:
                    with open(cache_file, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    print(f"📦 Données en cache (âge: {int(file_age/3600):.1f}h)")
                    return data
                except Exception as e:
                    print(f"⚠️ Erreur lecture cache: {e}")
                    return None
            else:
                print(f"⏰ Cache expiré (âge: {int(file_age/3600):.1f}h)")
        
        return None
    
    def _save_to_cache(self, cache_key, data):
        cache_file = self._get_cache_filename(cache_key)
        
        try:
            with open(cache_file, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"💾 Données sauvegardées dans le cache")
        except Exception as e:
            print(f"⚠️ Erreur sauvegarde cache: {e}")
    
    def _make_request(self, endpoint, params, force_refresh=False):
        cache_key = self._get_cache_key(endpoint, params)
        
        # Vérifier le cache d'abord (sauf si force_refresh)
        if not force_refresh:
            cached_data = self._get_from_cache(cache_key)
            if cached_data is not None:
                return cached_data
        
        # Faire l'appel API
        url = f"{self.base_url}{endpoint}"
        response = requests.get(url, headers=self.headers, params=params)
        self.api_calls += 1
        print(f"🌐 Appel API #{self.api_calls}")
        
        if response.status_code == 200:
            data = response.json()
            self._save_to_cache(cache_key, data)
            return data
        else:
            print(f"❌ Erreur API: {response.status_code}")
        return None
    
    def get_teams(self, force_refresh=False):
        """Récupère la liste des équipes de la Ligue 1"""
        params = {'league': self.league_id, 'season': self.season}
        return self._make_request('/teams', params, force_refresh)
    
    def get_team_statistics(self, team_id, force_refresh=False):
        """Récupère les statistiques d'une équipe"""
        params = {
            'league': self.league_id,
            'season': self.season,
            'team': team_id
        }
        return self._make_request('/teams/statistics', params, force_refresh)
    
    def get_standings(self, force_refresh=False):
        """Récupère le classement de la Ligue 1"""
        params = {'league': self.league_id, 'season': self.season}
        return self._make_request('/standings', params, force_refresh)
    
    def get_head_to_head(self, team1_id, team2_id, last=10, force_refresh=False):
        """Récupère l'historique des confrontations"""
        params = {'h2h': f"{team1_id}-{team2_id}", 'last': last}
        return self._make_request('/fixtures/headtohead', params, force_refresh)
    
    def get_fixtures(self, date=None, team_id=None, force_refresh=False):
        """Récupère les matchs selon différents critères
        
        Args:
            date: Date au format YYYY-MM-DD (ex: '2026-02-08')
            team_id: ID de l'équipe pour filtrer les matchs
            force_refresh: Forcer le rafraîchissement du cache
        """
        params = {'league': self.league_id, 'season': self.season}
        
        if date:
            params['date'] = date
        if team_id:
            params['team'] = team_id
            
        return self._make_request('/fixtures', params, force_refresh)
    
    def clear_cache(self):
        """Vide le cache en supprimant tous les fichiers"""
        if os.path.exists(self.cache_dir):
            files = [f for f in os.listdir(self.cache_dir) if f.endswith('.json')]
            for file in files:
                os.remove(os.path.join(self.cache_dir, file))
            print(f"🗑️ Cache vidé ({len(files)} fichiers supprimés)")
        else:
            print("📁 Dossier cache inexistant")
    
    def get_cache_stats(self):
        """Affiche les statistiques du cache"""
        if os.path.exists(self.cache_dir):
            files = [f for f in os.listdir(self.cache_dir) if f.endswith('.json')]
            total_size = sum(os.path.getsize(os.path.join(self.cache_dir, f)) for f in files)
            
            print(f"📊 Statistiques du cache:")
            print(f"   - Fichiers en cache: {len(files)}")
            print(f"   - Taille totale: {total_size / 1024:.2f} KB")
            print(f"   - Appels API effectués: {self.api_calls}")
            print(f"   - Durée de validité: {self.cache_duration}s ({self.cache_duration/3600:.1f}h)")
            print(f"   - Dossier: {os.path.abspath(self.cache_dir)}")
        else:
            print(f"📁 Dossier cache inexistant")

# Initialisation de l'API avec cache 24h
api = FootballAPI(API_KEY, LIGUE1_ID, CURRENT_SEASON, cache_duration=86400)
print("✅ API initialisée avec cache 24h")

✅ API initialisée avec cache 24h


## 🗑️ (Optionnel) Vider le cache avant collecte

Décommentez pour forcer le rafraîchissement complet des données.

In [15]:
# ⚠️ ATTENTION : Cela forcera le rafraîchissement complet
api.clear_cache()
print("💡 Cache conservé. Décommentez la ligne ci-dessus pour forcer un rafraîchissement.")

🗑️ Cache vidé (26 fichiers supprimés)
💡 Cache conservé. Décommentez la ligne ci-dessus pour forcer un rafraîchissement.


## 📥 Étape 1 : Récupérer la liste des équipes

In [16]:
print("📥 Récupération de la liste des équipes...\n")

teams_data = api.get_teams()

if teams_data and teams_data.get('response'):
    teams = teams_data['response']
    
    print(f"\n✅ {len(teams)} équipes trouvées\n")
    
    # Créer un DataFrame pour visualisation
    teams_list = []
    for team in teams:
        teams_list.append({
            'ID': team['team']['id'],
            'Nom': team['team']['name'],
            'Code': team['team']['code'],
            'Stade': team['venue']['name']
        })
    
    df_teams = pd.DataFrame(teams_list)
    print("🏟️ Équipes de Ligue 1:\n")
    display(df_teams)
    
    # Sauvegarder les IDs pour la suite
    team_ids = [t['team']['id'] for t in teams]
    print(f"\n💾 {len(team_ids)} IDs d'équipes sauvegardés en mémoire")
else:
    print("❌ Erreur lors de la récupération des équipes")
    team_ids = []

📥 Récupération de la liste des équipes...



🌐 Appel API #1
💾 Données sauvegardées dans le cache

✅ 18 équipes trouvées

🏟️ Équipes de Ligue 1:



,ID,Nom,Code,Stade
0,77,Angers,ANG,Stade Raymond-Kopa
1,79,Lille,LIL,Decathlon Arena – Stade Pierre-Mauroy
2,80,Lyon,LYO,Groupama Stadium
3,81,Marseille,MAR,Stade Orange Vélodrome
4,83,Nantes,NAN,Stade de la Beaujoire - Louis Fonteneau
5,84,Nice,NIC,Allianz Riviera
6,85,Paris Saint Germain,PAR,Parc des Princes
7,91,Monaco,MON,Stade Louis-II
8,94,Rennes,REN,Roazhon Park
9,95,Strasbourg,STR,Stade de la Meinau



💾 18 IDs d'équipes sauvegardés en mémoire


## 📊 Étape 2 : Récupérer les statistiques de chaque équipe

⚠️ **Attention** : Cette cellule va faire **~20 appels API** (un par équipe)

In [17]:
if len(team_ids) > 0:
    print(f"📊 Récupération des statistiques de {len(team_ids)} équipes...\n")
    print("⏱️ Cela peut prendre quelques minutes...\n")
    
    stats_collected = 0
    stats_from_cache = 0
    
    for i, team_id in enumerate(team_ids, 1):
        # Récupérer les stats
        stats = api.get_team_statistics(team_id)
        
        if stats and stats.get('response'):
            stats_collected += 1
            team_name = stats['response']['team']['name']
            print(f"  [{i}/{len(team_ids)}] ✅ {team_name}")
        else:
            print(f"  [{i}/{len(team_ids)}] ❌ Équipe ID {team_id}")
        
        # Petit délai entre les requêtes pour ne pas surcharger l'API
        if i < len(team_ids):
            time.sleep(0.5)
    
    print(f"\n✅ Collecte terminée : {stats_collected}/{len(team_ids)} équipes")
    print(f"🌐 Appels API effectués : {api.api_calls}")
else:
    print("⚠️ Aucune équipe à traiter. Exécutez d'abord la cellule précédente.")

📊 Récupération des statistiques de 18 équipes...

⏱️ Cela peut prendre quelques minutes...

🌐 Appel API #2
💾 Données sauvegardées dans le cache
  [1/18] ✅ Angers
🌐 Appel API #3
💾 Données sauvegardées dans le cache
  [2/18] ✅ Lille
🌐 Appel API #4
💾 Données sauvegardées dans le cache
  [3/18] ✅ Lyon
🌐 Appel API #5
💾 Données sauvegardées dans le cache
  [4/18] ✅ Marseille
🌐 Appel API #6
💾 Données sauvegardées dans le cache
  [5/18] ✅ Nantes
🌐 Appel API #7
💾 Données sauvegardées dans le cache
  [6/18] ✅ Nice
🌐 Appel API #8
💾 Données sauvegardées dans le cache
  [7/18] ✅ Paris Saint Germain
🌐 Appel API #9
💾 Données sauvegardées dans le cache
  [8/18] ✅ Monaco
🌐 Appel API #10
💾 Données sauvegardées dans le cache
  [9/18] ✅ Rennes
🌐 Appel API #11
💾 Données sauvegardées dans le cache
  [10/18] ❌ Équipe ID 95
🌐 Appel API #12
💾 Données sauvegardées dans le cache
  [11/18] ❌ Équipe ID 96
🌐 Appel API #13
💾 Données sauvegardées dans le cache
  [12/18] ❌ Équipe ID 97
🌐 Appel API #14
💾 Données sauveg

## 🏆 Étape 3 : Récupérer le classement

In [18]:
print("🏆 Récupération du classement...\n")

standings = api.get_standings()

if standings and standings.get('response'):
    teams_data = []
    for team in standings['response'][0]['league']['standings'][0]:
        teams_data.append({
            'Rang': team['rank'],
            'Équipe': team['team']['name'],
            'Points': team['points'],
            'Joués': team['all']['played'],
            'V-N-D': f"{team['all']['win']}-{team['all']['draw']}-{team['all']['lose']}",
            'Diff': team['goalsDiff']
        })
    
    df_standings = pd.DataFrame(teams_data)
    print("✅ Classement récupéré\n")
    display(df_standings)
else:
    print("❌ Erreur lors de la récupération du classement")

🏆 Récupération du classement...

🌐 Appel API #20
💾 Données sauvegardées dans le cache
✅ Classement récupéré



,Rang,Équipe,Points,Joués,V-N-D,Diff
0,1,Lens,49,21,16-1-4,20
1,2,Paris Saint Germain,48,20,15-3-2,27
2,3,Lyon,42,21,13-3-5,14
3,4,Marseille,39,20,12-3-5,24
4,5,Lille,33,21,10-3-8,4
5,6,Strasbourg,31,21,9-4-8,8
6,7,Toulouse,31,21,8-7-6,8
7,8,Rennes,31,21,8-7-6,-3
8,9,Monaco,28,21,8-4-9,-1
9,10,Lorient,28,21,7-7-7,-6


## 📊 Résumé de la collecte

In [23]:
print("="*60)
print("📊 RÉSUMÉ DE LA COLLECTE DE DONNÉES")
print("="*60)

api.get_cache_stats()

print(f"\n⏰ Données collectées le : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"\n💡 Ces données sont maintenant disponibles dans le cache pour :")
print(f"   - predictions_ligue1.ipynb (Prédictions simples)")
print(f"   - 2_predictions_stats_avancees.ipynb (Prédictions stats avancées)")
print(f"\n📅 Prochaine collecte recommandée :")
print(f"   - Après les matchs de la prochaine journée")
print(f"   - Dans environ 24h pour données fraîches")

print("\n✅ Collecte terminée avec succès !")

📊 RÉSUMÉ DE LA COLLECTE DE DONNÉES
📊 Statistiques du cache:
   - Fichiers en cache: 26
   - Taille totale: 147.72 KB
   - Appels API effectués: 26
   - Durée de validité: 86400s (24.0h)
   - Dossier: c:\Users\nboud\Documents\GitHub-My-Projects\predictions-football-fr-python\cache

⏰ Données collectées le : 2026-02-08 21:47:33

💡 Ces données sont maintenant disponibles dans le cache pour :
   - predictions_ligue1.ipynb (Prédictions simples)
   - 2_predictions_stats_avancees.ipynb (Prédictions stats avancées)

📅 Prochaine collecte recommandée :
   - Après les matchs de la prochaine journée
   - Dans environ 24h pour données fraîches

✅ Collecte terminée avec succès !


## 🔍 Vérification du cache

Lister tous les fichiers du cache.

In [21]:
import glob

cache_files = glob.glob('cache/*.json')

print(f"📁 Fichiers dans le cache : {len(cache_files)}\n")

if cache_files:
    # Trier par date de modification (plus récent d'abord)
    cache_files.sort(key=os.path.getmtime, reverse=True)
    
    print("🕐 Derniers fichiers modifiés :\n")
    for i, filepath in enumerate(cache_files[:10], 1):
        filename = os.path.basename(filepath)
        size = os.path.getsize(filepath) / 1024
        mod_time = datetime.fromtimestamp(os.path.getmtime(filepath))
        age_hours = (datetime.now() - mod_time).total_seconds() / 3600
        
        print(f"  {i}. {filename[:16]}... ({size:.1f} KB) - {age_hours:.1f}h")
    
    if len(cache_files) > 10:
        print(f"\n  ... et {len(cache_files) - 10} autres fichiers")
else:
    print("⚠️ Aucun fichier dans le cache")

📁 Fichiers dans le cache : 26

🕐 Derniers fichiers modifiés :

  1. 8f163737c4f2ebdb... (8.6 KB) - 0.0h
  2. 0f8d17dd84b06ebe... (8.6 KB) - 0.0h
  3. fdb9c9b093deaa35... (8.6 KB) - 0.0h
  4. 97097004f8436d52... (8.7 KB) - 0.0h
  5. 1ac1d51a6b7feb72... (8.6 KB) - 0.0h
  6. 9a1f8cba129482b0... (8.6 KB) - 0.0h
  7. 717a4249268326e3... (23.4 KB) - 0.0h
  8. 13f7063be1fa22ac... (0.3 KB) - 0.0h
  9. d7fc7a90082e982d... (0.3 KB) - 0.0h
  10. 8639c41c0f404fb6... (0.3 KB) - 0.0h

  ... et 16 autres fichiers


---

## 💡 Notes d'utilisation

### ✅ Quand exécuter ce notebook ?

1. **Première utilisation** : Collecter toutes les données initiales
2. **Avant chaque journée** : Rafraîchir les statistiques
3. **Après les matchs** : Mettre à jour avec les derniers résultats

### 🔄 Gestion du cache

- **Durée** : 24h par défaut
- **Persistence** : Les données restent disponibles même après fermeture
- **Rafraîchissement** : Décommentez `api.clear_cache()` pour forcer

### 📊 Consommation API

**Première exécution complète** :
- Liste équipes : 1 appel
- Statistiques : ~20 appels
- Classement : 1 appel
- Head-to-head : ~6 appels
- **Total : ~28 appels** (28% du quota quotidien)

**Exécutions suivantes** :
- Si cache valide : 0 appel API ! 🎉
- Si cache expiré : Rafraîchissement automatique

### 🎯 Prochaines étapes

1. ✅ Données collectées et en cache
2. ➡️ Allez dans `predictions_ligue1.ipynb` ou `2_predictions_stats_avancees.ipynb`
3. 🚀 Exécutez les prédictions (sans appels API supplémentaires !)

---

**Auteur** : Système de prédiction Ligue 1  
**Version** : 1.0  
**Date** : Février 2026

## ⚽ Récupérer les prochains matchs
# Récupérer les matchs du jour

In [28]:
today = datetime.now().strftime("%Y-%m-%d")
fixtures = api.get_fixtures(date=today)

if fixtures and fixtures['response']:
    print(f"📅 Matchs du {today}\n")
    for match in fixtures['response']:
        home = match['teams']['home']['name']
        away = match['teams']['away']['name']
        date_time = match['fixture']['date']
        status = match['fixture']['status']['long']
        
        print(f"🏟️  {home} vs {away}")
        print(f"   ⏰ {date_time} - {status}\n")
else:
    print(f"Aucun match prévu le {today}")
    print("\n💡 Astuce: Modifiez la date pour voir d'autres matchs")

🌐 Appel API #1
💾 Données sauvegardées dans le cache
📅 Matchs du 2026-02-08

🏟️  Paris Saint Germain vs Marseille
   ⏰ 2026-02-08T19:45:00+00:00 - Halftime

🏟️  Nice vs Monaco
   ⏰ 2026-02-08T14:00:00+00:00 - Match Finished

🏟️  Angers vs Toulouse
   ⏰ 2026-02-08T16:15:00+00:00 - Match Finished

🏟️  Auxerre vs Paris FC
   ⏰ 2026-02-08T16:15:00+00:00 - Match Finished

🏟️  Le Havre vs Strasbourg
   ⏰ 2026-02-08T16:15:00+00:00 - Match Finished



## 📊 Récupérer le classement de la Ligue 1
# Récupérer le classement

In [29]:
standings = api.get_standings()

if standings and standings['response']:
    teams_data = []
    for team in standings['response'][0]['league']['standings'][0]:
        teams_data.append({
            'Rang': team['rank'],
            'Équipe': team['team']['name'],
            'Points': team['points'],
            'Joués': team['all']['played'],
            'Victoires': team['all']['win'],
            'Nuls': team['all']['draw'],
            'Défaites': team['all']['lose'],
            'BP': team['all']['goals']['for'],
            'BC': team['all']['goals']['against'],
            'Diff': team['goalsDiff'],
            'Forme': team.get('form', '-')  # Utiliser '-' si la forme n'est pas disponible
        })
    
    df_standings = pd.DataFrame(teams_data)
    print("🏆 Classement Ligue 1")
    display(df_standings)
else:
    print("❌ Impossible de récupérer le classement")
# Confrontations PSG (85) vs Marseille (81)
team1_id = 85  # PSG
team2_id = 81  # Marseille

h2h = api.get_head_to_head(team1_id, team2_id, last=5)

if h2h and h2h['response']:
    print(f"🔄 Dernières confrontations\n")
    
    for match in h2h['response']:
        home = match['teams']['home']['name']
        away = match['teams']['away']['name']
        score_home = match['goals']['home']
        score_away = match['goals']['away']
        date = match['fixture']['date'][:10]
        
        print(f"📅 {date}: {home} {score_home} - {score_away} {away}")
    
    # Statistiques globales
    team1_wins = sum(1 for m in h2h['response'] if 
                     (m['teams']['home']['id'] == team1_id and m['goals']['home'] > m['goals']['away']) or
                     (m['teams']['away']['id'] == team1_id and m['goals']['away'] > m['goals']['home']))
    
    team2_wins = sum(1 for m in h2h['response'] if 
                     (m['teams']['home']['id'] == team2_id and m['goals']['home'] > m['goals']['away']) or
                     (m['teams']['away']['id'] == team2_id and m['goals']['away'] > m['goals']['home']))
    
    draws = len(h2h['response']) - team1_wins - team2_wins
    
    print(f"\n📊 Bilan: {team1_wins} victoires - {draws} nuls - {team2_wins} défaites")
else:
    print("❌ Impossible de récupérer l'historique")

📦 Données en cache (âge: 0.0h)
🏆 Classement Ligue 1


,Rang,Équipe,Points,Joués,Victoires,Nuls,Défaites,BP,BC,Diff,Forme
0,1,Lens,49,21,16,1,4,37,17,20,WWLWW
1,2,Paris Saint Germain,48,20,15,3,2,43,16,27,WWWWW
2,3,Lyon,42,21,13,3,5,34,20,14,WWWWW
3,4,Marseille,39,20,12,3,5,46,22,24,DWWLW
4,5,Lille,33,21,10,3,8,34,30,4,DLLLL
5,6,Strasbourg,31,21,9,4,8,34,26,8,LWWDD
6,7,Toulouse,31,21,8,7,6,31,23,8,DWWLW
7,8,Rennes,31,21,8,7,6,31,34,-3,LLLDW
8,9,Monaco,28,21,8,4,9,32,33,-1,DWDLL
9,10,Lorient,28,21,7,7,7,27,33,-6,LWWWD


📦 Données en cache (âge: 0.0h)
🔄 Dernières confrontations

📅 2026-02-08: Paris Saint Germain 2 - 0 Marseille
📅 2026-01-08: Paris Saint Germain 2 - 2 Marseille
📅 2025-09-22: Marseille 1 - 0 Paris Saint Germain
📅 2025-03-16: Paris Saint Germain 3 - 1 Marseille
📅 2024-10-27: Marseille 0 - 3 Paris Saint Germain

📊 Bilan: 3 victoires - 1 nuls - 1 défaites


## 🚀 Prochaines étapes

Pour améliorer les prédictions, vous pouvez :

1. **Collecter plus de données** : xG (expected goals), possession, tirs cadrés, etc.
2. **Machine Learning** : Utiliser scikit-learn pour entraîner un modèle
3. **Facteurs supplémentaires** : 
   - Avantage du terrain (domicile/extérieur)
   - Blessures et suspensions
   - Météo
   - Cotes des bookmakers
4. **Validation** : Tester les prédictions sur des données historiques

### 📚 IDs d'équipes principales de Ligue 1

- PSG: 85
- Marseille: 81
- Monaco: 91
- Lyon: 80
- Lille: 79
- Rennes: 94
- Nice: 33
- Lens: 116

## 📊 Comparaison des Approches

Le projet propose 3 approches de prédiction :

| Critère | Simple (ce notebook) | Stats Avancées | ML |
|---------|---------------------|----------------|----|
| **Fichier** | `1_predictions_simples.ipynb` | `2_predictions_stats_avancees.ipynb` | `3_predictions_ml.ipynb` |
| **Entraînement** | ❌ Non | ❌ Non | ✅ Oui |
| **Domicile/Ext** | ❌ Non | ✅ Oui | ✅ Oui |
| **Head-to-head** | ❌ Non | ✅ Oui | ✅ Possible |
| **Transparence** | ✅ Totale | ✅ Totale | ❌ Boîte noire |
| **Facteurs** | 3 | 6 | 20+ |
| **Temps calcul** | <1s | <5s | 10-15min |
| **Précision** | ~55-60% | ~60-65% | ~65-70% |

## 📚 Documentation

- 📖 [Méthodologie de l'approche simple](docs/prediction-simple.md)
- 📊 [Documentation du projet](docs/README.md)
- 🔗 [API-Football](https://www.api-football.com/)

---

**Auteur** : Système de prédiction Ligue 1  
**Version** : 1.0 - Prédictions Simples  
**Date** : Février 2026